# Investment and Trading Capstone Project
## Build a Stock Price Indicator

## 1. Introduction
Financial institutions around the world are trading in billions of dollars on a daily basis. Investment firms, hedge funds and even individuals have been using financial models to better understand market behavior and make profitable investments and trades.  A wealth of information is available in the form of historical stock prices and company performance data, suitable for machine learning algorithms to process.

In this project, I will be building a stock price predictor that takes daily trading data over a certain date range as input, and outputs projected estimates for given query dates. Note that the inputs will contain multiple metrics, such as opening price (Open), highest price the stock traded at (High), how many stocks were traded (Volume) and closing price adjusted for stock splits and dividends (Adjusted Close); I will be predicting the Adjusted Close price.

### Install and import relevant libraries

In [ ]:
#this is the colab code: https://colab.research.google.com/drive/1Xhcgrryu1a9eknPVIM3r0QAHUDW8_Zwn?usp=sharing

In [ ]:
# imports

import numpy as np
import pandas as pd
#from pandas_datareader.data import DataReader
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython import display
import seaborn as sns
%matplotlib inline

#import functions
#import plotting

from datetime import datetime,date

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.metrics import mean_squared_error
from keras.layers import LSTM

# Needed to help our plots look cleaner with plotly 
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
init_notebook_mode(connected=True)


## 2. Exploratory Analysis
In this section we would be getting the data, view the data and compute some statistics and plots to understand the data very well before we start the modelling process.
### Getting the Data
The data I will be using for the project is from [yahoo finance](https://finance.yahoo.com/).

In [ ]:
# https://towardsdatascience.com/free-stock-data-for-python-using-yahoo-finance-api-9dafd96cad2e
# https://www.learndatasci.com/tutorials/python-finance-part-yahoo-finance-api-pandas-matplotlib/

# Install yfinance
#!pip install yfinance

In [ ]:
# Download stock data then export as CSV

import yfinance as yfin

#tickes are S&P 500, Apple, Microsoft, Amazon, Johnson & Johnson, Pfizer, Ford and Google
tickers = ("SPY", "AAPL", "MSFT", "AMZN", "F", "GOOGL") 

start = "2001-01-01"
end = '2021-08-17'

fin_data = yfin.download(tickers, start, end) #download yahoo finance data for specific dates

fin_data.to_csv('fin_data.csv') #convert data to csv

### View the Data

In [ ]:
#check the dimensions of the data
fin_data.shape

In [ ]:
#view the first 5 rows of the data
fin_data.head()

In [ ]:
#view the last 5 values of the data
fin_data.tail()

In [ ]:
#check if there are missing values for each type of stock
fin_data.isnull().sum()

Google has some missing values, and we will use the `fillna` method to resolve the missing values.

In [ ]:
# handing missing values
fin_data.fillna(method='ffill', inplace = True) # use front fill method
fin_data.fillna(method='bfill', inplace = True) # use back fill method

In [ ]:
#check to see if there are still any missing values
fin_data.isnull().sum()

Finally, we have no missing values. We will now check some descriptive statistics of the data.

In [ ]:
#view descriptive statistics of adjusted close process of the stocks
fin_data[['Adj Close']].describe()

In [ ]:
# view general info
fin_data.info()

In [ ]:
# View the maximum close date of stocks
def max_close(stocks,df):
    """ This calculates and returns the maximum closing value of a specific stock"""
    return df['Close'][stocks].max() # computes and returns the maximum closing stock value

In [ ]:
# test the above function with specific stocks
def test_max():
    """ This tests the max_close function"""
    for stocks in ["SPY", "AAPL", "MSFT", "AMZN", "F", "GOOGL"]:
        print("Maxiumum Closing Value for {} is {}".format(stocks, max_close(stocks,fin_data)))

test_max()        
#if __name__ == "__main__" :
#    test_()

In [ ]:
# calculate the mean volume for the stocks
def mean_vol(stocks,df):
    """ This calculates and returns the minimum volume of a specific stock"""
    return df['Volume'][stocks].mean() # computes and returns the minimum volume of a stock

In [ ]:
# test the above function with specific stocks
def test_mean():
    """ This tests the max_close function"""
    for stocks in ["SPY", "AAPL", "MSFT", "AMZN", "F", "GOOGL"]:
        print("Mean Volume for {} is {}".format(stocks, mean_vol(stocks,fin_data)))

test_mean()        

### Plotting stock prices

In [ ]:
# Plot function for the Adjusted closing value
def plot_adj(df,title,stocks,y=0):
        ax = df['Adj Close'][stocks].plot(title=title, figsize=(16,8), ax=None)
        ax.set_xlabel("Date")
        ax.set_ylabel("Stock Price")
        ax.axhline(y=y,color='black')
        ax.legend(stocks, loc='upper left')
        plt.show()

In [ ]:
# View the plot of Adjusted close
stocks = ["SPY", "AAPL", "MSFT", "AMZN",  "F", "GOOGL"]
plot_adj(fin_data,"Adjusted Close Stock Prices",stocks)

From the plot above, we can see that Amazon and Google has the highest growth. This could be as a result of the increase in useage of these companies world wide, but let us normalize the data before we make that conclusion. I will explore more features in the dataset before normalizing it.

In [ ]:
# Plot function for the High prices
def plot_high(df,title,stocks,y=0):
        ax = df['High'][stocks].plot(title=title, figsize=(16,8), ax = None)
        ax.set_xlabel("Date")
        ax.set_ylabel("Stock Price")
        ax.axhline(y=y,color='black')
        ax.legend(stocks, loc='upper left')
        plt.show()

In [ ]:
# View the plot of high stock prices
stocks = ["SPY", "AAPL", "MSFT", "AMZN",  "F", "GOOGL"]
title = "High Stock Prices"
plot_high(fin_data,title,stocks)

Similar to the adjusted value plot, we can see high stock prices for Amazon and Google. 

In [ ]:
#view only google Adjusted close price
fin_data['Adj Close']['GOOGL']

In [ ]:
# view only google and amazon Adjusted close price
fin_data['Adj Close'][['GOOGL', 'AMZN']]

In [ ]:
# view all stocks adjusted close prices for amazon and google from Jan 2010 to Aug 2021
fin_data['Adj Close'].loc['2010-01-01':'2021-08-17', ['AMZN', 'GOOGL']]

In [ ]:
# create function to plot data
def plot_data1(df,stocks,title,ylabel='Stock Price',y=0):
    """This funtion plots stock prices"""
    ax = df.plot(title=title, figsize=(16,8), ax=None, fontsize=2)
    ax.set_xlabel("Date")
    ax.set_label(ylabel)
    ax.axhline(y=y,color='black')
    ax.legend(stocks, loc='upper left')
    plt.show()

In [ ]:
# create function to plot selected stocks
def selected_plot(df, columns,stocks, start_idx, end_idx):
    """This function plots specific stocks over a given date range"""
    plot_data1(df[columns].loc[start_idx:end_idx, stocks],stocks, title="Plot for selected Stocks")

In [ ]:
# create function to plot data based on specific columns, symbols, and date ranges
def test_select():
    """This function plots stock data based on specific columns, symbols, and date ranges """
    # specify columns to plot and stock symbols
    columns = 'Adj Close'
    stocks = ['GOOGL', 'AMZN']  
        
    # Get stock data
    df = fin_data

    # Slice and plot
    selected_plot(df, columns, stocks, '2010-01-01', '2021-08-17')
    
test_select()  # run the plot function  

### Normalizing the data
We want to know how the different types of stocks went up and down with respect to the others. In order to do this, we will normalize the data. We do this by dividing the values of each column by day one to ensure that each stock starts with ${$1}$.

In [ ]:
def plot_data2(df,stocks,title='Stock Prices',ylabel="Stock Price",y=0, start='2001-01-01', end ='2021-08-17'):
    
    """ This function creates a plot of adjusted close stock prices
    inputs:
    df - dataframe
    title - plot title
    stocks - the stock symbols of each company
    ylabel - y axis label
    y - horizontal line(integer)
    output: the plot of adjusted close stock prices
    """
    df_new = df[start:end]
    #ax = df_new['Adj Close'][stocks].plot(title=title, figsize=(16,8), ax = None)
    ax = df_new.plot(title=title, figsize=(16,8), ax = None)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    ax.axhline(y=y,color='black')
    ax.legend(stocks, loc='upper left')
    plt.show()

In [ ]:
# create function that normalizes the data
def normalize_data(df):
    """ 
    This function normalizes the stock prices using the first row of the dataframe
    input - stock data
    output - normalized stock data
    """
    return df/df.iloc[0,:]    

In [ ]:
# plot the data with the new normalized data

stocks = ["SPY", "AAPL", "MSFT", "AMZN", "F", "GOOGL"]

plot_data2(normalize_data(fin_data['Adj Close'][stocks]),stocks,title = "Normalized Stock Prices", ylabel = 'Cummulative return',y=1)

From the above cummulative return plot, we can see that Apple has the highest return over the years, while Amazon was second and Google third and Microsoft fourth. The growth of Google and Microsoft looks much more stable than Apple and Amazon. Looking at the plot, Apple has alot of volatility and risky stocks especially in recent years.

### Cumulative Return
Let's have a look at how the pandemic affected stock prices for these companies.

In [ ]:
stocks = ["SPY", "AAPL", "MSFT", "AMZN", "F", "GOOGL"]

plot_data2(normalize_data(fin_data['Adj Close'][stocks]['2019-01-01':'2019-12-31']), stocks,title = '2019', ylabel = 'Cummulative return',y=1, start='2019-01-01', end = '2019-12-31') #2019
plot_data2(normalize_data(fin_data['Adj Close'][stocks]['2020-01-01':'2020-12-31']), stocks,title = '2020', ylabel = 'Cummulative return',y=1, start='2020-01-01', end = '2020-12-31') #2020
plot_data2(normalize_data(fin_data['Adj Close'][stocks]['2021-01-01':'2021-08-17']), stocks,title = '2021', ylabel = 'Cummulative return',y=1, start='2021-01-01', end = '2021-08-17') #2021

From the above plots, let's take note of the following:

    1. 2019: Before the pandemic, we notice that most of the companies stocks were doing relatively well with Apple and Microsoft taking the lead and Pfizer trailing behind.
    2. 2020: On the onset of the pandemic around Spring, we notice that there was a fall in stock prices for all the companies, but afterwards the technology companies like Amazon, Apple, Microsoft and Google started to grow again. But companies like Pfizer, Ford and S&P 500 did not do very well especially Ford.
    3. 2021: As the vaccine rollout began and the lockdown began to be lifted, we can see significant growth in the stock prices of Ford in particular given that its stock prices which were low in 2020 due to the pandemic. Companies like Google and Microsoft and S&P 500 also grew. Overall there was an improvement in the stock prices of all the companies we considered.

### Computing the Rolling mean and Bollinger Bands
The rolling mean may give us some idea about the true underlying prices of a stock. If there is a significant deviation below or above the rolling mean, it may give us an idea about a potential buying and selling opportunity respectively. The challenge remains to know when this deviation is significant enough to pay attention to it. [Bollinger Bands](https://en.wikipedia.org/wiki/Bollinger_Bands) is a statistical chart that contains the volatility of a financial instrument over time. `Bollinger` observed that looking at the recent volatility of the stock, if it is very volatile, we might discard the movement above and below the mean. But if it is not very volatile we may want to pay attention to it. He added a band  $2\delta$ (2 standard deviations)  above and below the mean. We would use the rolling standard deviation to help us achieve this.

In [ ]:
# compute rolling mean, rolling standard deviation, upper and lower bands

def rolling_stats(df, stocks, type_, window = 20):
    """
    This function computes the rolling mean and Bollinger bands
    inputs : 
    df - dataframe
    stocks - the type of stocks we would be analyzing
    type_ - the price type of the rolling calculation
    window - number of days used to calculate the statistics
    output: 
    rolling mean, rolling standard deviation, upper and lower bands of 2 std each
    """
    
    val = df[(type_,stocks)]
    rolling_mean = df[(type_, stocks)].rolling(window=window).mean()
    rolling_std = df[(type_, stocks)].rolling(window=window).std()
    upper_band = rolling_mean + rolling_std*2
    lower_band = rolling_mean - rolling_std*2
    
    return val, rolling_mean, rolling_std, upper_band, lower_band
    

In [ ]:
# plot the rolling mean, rolling standard deviation, upper and lower bands

def rolling_plot(stocks, val, rolling_mean, upper_band, lower_band, title='Rolling mean'):
    """
    This function plots the rolling mean and Bollinger bands
    inputs : 
    stocks - the type of stocks we would be analyzing
    val - value of the stock price type
    rolling_mean - rolling mean
    upper_band - stocks upper band
    lower_band - stocks lower band
    title - plot title
    output: 
    plot of rolling mean, rolling standard deviation, upper and lower bands of 2 std each
    """
    
    ax = rolling_mean.plot(title=title, figsize=(16,8), label='Rolling Mean')
    plt.plot(upper_band, label = 'Upper Band')
    plt.plot(lower_band, label = 'Lower Band')
    plt.plot(val, label = 'Value of Stock')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend(loc='upper left')
    plt.show()
    
    return ax


In [ ]:
stocks = 'MSFT'
type_ = 'Adj Close'

val, rolling_mean, rolling_std, upper_band, lower_band = rolling_stats(fin_data['2019-01-01':'2021-08-17'], stocks, type_)

rolling_plot(stocks, val, rolling_mean, upper_band, lower_band, title='Rolling mean of {} for {} 20 days window'.format(type_,stocks))
plt.show()

In [ ]:
# view rolling statistics for Apple
stocks = 'AAPL'
type_ = 'Adj Close'

val, rolling_mean, rolling_std, upper_band, lower_band = rolling_stats(fin_data['2019-01-01':'2021-08-17'], stocks, type_)

rolling_plot(stocks, val, rolling_mean, upper_band, lower_band, title='Rolling mean of {} for {} 20 days window'.format(type_,stocks))
plt.show()

In [ ]:
# view rolling statistics for Ford
stocks = 'F'
type_ = 'Adj Close'

val, rolling_mean, rolling_std, upper_band, lower_band = rolling_stats(fin_data['2019-01-01':'2021-08-17'], stocks, type_)

rolling_plot(stocks, val, rolling_mean, upper_band, lower_band, title='Rolling mean of {} for {} 20 days window'.format(type_,stocks))
plt.show()

From the above plots, we can see that the initial values for the rolling mean are missing. This is as a result of the 20 days window period we used at the beginning which had no values. We can also observe that the rolling mean follows the movement of the raw stock prices and it is less spiky. We can also see that Ford has lower stock prices than Microsoft and Apple in 2020. In this data exploration session, I only considered a 20 days window and for 2 standard deviation away from the mean for simplicity. You may want to try computing different windows sizes and standard deviations to see how differently the stock prices behave.

### Computing Daily Returns
Daily returns tells us how much the stock price go up and down on a particular day. We can compute using the following function
$$
DailyReturn = \frac{price(t)}{price(t-1)} -1
$$
where `price(t)` is the price of today's stock and `price(t-1)` is the price of yesterday's stock.

In [ ]:
def daily_returns_cal(df,stocks):
    """
    This function computes and returns the daily return values
    input: df (dataframe) and stocks
    output: daily return values
    """
    
    daily_returns = (df[('Adj Close', stocks)][1:]/df[('Adj Close', stocks)][:-1].values) - 1
       
    return daily_returns

In [ ]:
# Daily return of Microsoft
plot_data2(daily_returns_cal(fin_data,'MSFT'),stocks=['MSFT'], ylabel = 'Daily returns',title='Stock Prices for Microsoft',y=0)

In [ ]:
# Daily return of Ford
plot_data2(daily_returns_cal(fin_data,'F'),stocks=['F'], ylabel = 'Daily returns',title='Stock Prices for Ford',y=0)

From the above plots, we can see that the volatility range for Ford is higher than Microsoft. This could be as a result of technology companies like Microsoft bouncing back faster during the pandemic.

## 3. Modelling
In this section I will be trying out some models to predict the Adjusted closing price of a stock.

### Predicting Adjusted close value of Microsoft stocks


In [ ]:
# Function that get specific stock data and fills in any missing value
def get_data(df, stocks):
    """
    This function gets a specific stock data and fills in any missing values using the fill forward and fill backward methods
    Input: 
    df - dataframe
    stocks - the type of stock
    Output - a cleaned dataset to be used for prediction
    """
    df1 = pd.DataFrame (data = df.iloc[:, df.columns.get_level_values(1)==stocks].values,
                          index = df.iloc[:, df.columns.get_level_values(1)==stocks].index,
                          columns = df.iloc[:, df.columns.get_level_values(1)==stocks].columns.get_level_values(0))
    
    df1.fillna(method='ffill', inplace= True)
    df1.fillna(method='bfill', inplace=True)
    
    return df1

In [ ]:
# get Microsoft data and view the first 5 rows
msft_data = get_data(fin_data, 'MSFT')  
msft_data.head()

In [ ]:
# plot showing Microsoft historical Adjusted closing prices
plt.figure(figsize=(16,6))
plt.title('Microsoft Adjusted Close Price History')
plt.plot(msft_data['Adj Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Adjusted Close Price USD ($)', fontsize=18)
plt.show()

#### Predicting using Long Short-Term Memory (LSTM) 
[LSTM](https://en.wikipedia.org/wiki/Long_short-term_memory#:~:text=Long%20short%2Dterm%20memory%20(LSTM)%20is%20an%20artificial%20recurrent,networks%2C%20LSTM%20has%20feedback%20connections.&text=A%20common%20LSTM%20unit%20is,gate%20and%20a%20forget%20gate.) is an artificial recurrent neural network (RNN) architecture used in deep learning that is capable of learning long-term dependencies. It processes data passing on information as it propagates forward and have a chain like structure. 

In [ ]:
#view the shape
msft_data.shape

In [ ]:
# create the variables for prediction and split into training and test sets

y = np.log(msft_data['Adj Close'].astype(int)) # we want to predict the adjusted close price
X = msft_data.drop('Adj Close', axis=1) # predictive variables (removing Adj close from it)

#split the data into training and test sets
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Build the LSTM model for Microsoft stock
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
# view model summary
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

#### Initial parameters used for LSTM

In [ ]:
# Train the model - part 1
model.fit(np.array(xtrain).reshape(-1,5,1), ytrain, batch_size =1, epochs=5, verbose=0)

In [ ]:
#predictions 
predictions = model.predict(np.array(xtest).reshape(-1,5,1))

In [ ]:
#calculation of the mean absolute error
mean_abs_error3 = mean_absolute_error(ytest, predictions)
mean_abs_error3

In [ ]:
# Train the model - part 2
model.fit(np.array(xtrain).reshape(-1,5,1), ytrain, batch_size =100, epochs=10, verbose=0)

In [ ]:
#predictions 2
predictions = model.predict(np.array(xtest).reshape(-1,5,1))

In [ ]:
#calculation of the mean absolute error 2
mean_abs_error3 = mean_absolute_error(ytest, predictions)
mean_abs_error3

### LSTM Refinement
#### Final Parameters used to tune LSTM

In [ ]:
# Build the LSTM model with the relu activation function
model2 = Sequential()
model2.add(LSTM(128, activation='relu', return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model2.add(LSTM(64, activation='relu', return_sequences=False))
model2.add(Dense(25))
model2.add(Dense(1))

In [ ]:
# Compile the model
model2.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# view model2 summary
model2.summary()

In [ ]:
# Train the model - part 3
model2.fit(np.array(xtrain).reshape(-1,5,1), ytrain, batch_size =1, epochs=5, verbose=0)

In [ ]:
#predictions 3
predictions2 = model2.predict(np.array(xtest).reshape(-1,5,1))

In [ ]:
#calculation of the mean absolute error 2
mean_abs_error2 = mean_absolute_error(ytest, predictions)
mean_abs_error2

In [ ]:
# Train the model - part 4 (increasing no. of epochs and batch_size)
model2.fit(np.array(xtrain).reshape(-1,5,1), ytrain, batch_size =100, epochs=10, verbose=0)

In [ ]:
#predictions 3
predictions2 = model2.predict(np.array(xtest).reshape(-1,5,1))

In [ ]:
#calculation of the mean absolute error 2
mean_abs_error2 = mean_absolute_error(ytest, predictions)
mean_abs_error2

In [ ]:
#plot showing the prediction and actual values

fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=80)
axs[0].set_title('Predicted vs actual values distribution')
ax1 = sns.kdeplot(data=ytest, color="g", label='Actual values',ax=axs[0])
ax2 = sns.kdeplot(data=predictions2, color="b", label='Predicted values', ax=ax1)
    
sns.regplot(x=ytest, y=predictions2)
plt.title('Predicted vs actual values distribution')
plt.xlabel('Stock Price')
#plt.legend()
ax1.legend()
plt.show()
plt.close()
    
print("Mean absolute error of {0}: {1}".format(model,mean_abs_error3))

In [ ]:
# Train the model part 5 (no activation function)
model.fit(np.array(xtrain).reshape(-1,5,1), ytrain, batch_size =800, epochs=50, verbose=0)

In [ ]:
#predictions 
predictions = model.predict(np.array(xtest).reshape(-1,5,1))

In [ ]:
#calculation of the mean absolute error
mean_abs_error3 = mean_absolute_error(ytest, predictions)
mean_abs_error3

In [ ]:
#plot showing the prediction and actual values

fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=80)
axs[0].set_title('Predicted vs actual values distribution')
ax1 = sns.kdeplot(data=ytest, color="g", label='Actual values',ax=axs[0])
ax2 = sns.kdeplot(data=predictions, color="b", label='Predicted values', ax=ax1)
    
sns.regplot(x=ytest, y=predictions)
plt.title('Predicted vs actual values distribution')
plt.xlabel('Stock Price')
#plt.legend()
ax1.legend()
plt.show()
plt.close()
    
print("Mean absolute error of {0}: {1}".format(model,mean_abs_error3))

From the above, we can see that the model is trained and evaluated using `LSTM`. The predicted and actual adjustable stock prices plots look are relatively similar with a mean absolute error of 0.0591 which isn't bad. However, there is still room for improvement and trying out other models to compare.

#### Predicting using Linear Regression

In [ ]:
#view the shape
msft_data.tail()

In [ ]:
# Building the model for Adj close prediction

y = np.log(msft_data['Adj Close'].astype(int)) # we want to predict the adjusted close price
X = msft_data.drop('Adj Close', axis=1) # predictive variables (removing Adj close from it)

#split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("x_train", X_train.shape)
print("y_train", y_train.shape)
print("x_test", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
#creating an instance of a Linear Regressor 
model_lin = LinearRegression()

#fit the model
model_lin.fit(X_train,y_train)

In [ ]:
# check the score, coef_ and intercept_ of the model
model_lin.score(X_train,y_train)
model_lin.coef_
model_lin.intercept_

print('The score of the model is {}, the coeficients  are {} and the intercept is {}'.format(model_lin.score(X_train,y_train),model_lin.coef_,model_lin.intercept_))

In [ ]:
#prediction
predict = model_lin.predict(X_test)
predict #view some predictions

In [ ]:
#calculation of the mean absolute error
mean_abs_error = mean_absolute_error(y_test, predict)
mean_abs_error

In [ ]:
#plot showing the prediction and actual values

fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=80)
axs[0].set_title('Predicted vs actual values distribution')
ax1 = sns.kdeplot(data=y_test, color="g", label='Actual values',ax=axs[0])
ax2 = sns.kdeplot(data=predict, color="b", label='Predicted values', ax=ax1)
    
sns.regplot(x=y_test, y=predict)
plt.title('Predicted vs actual values distribution')
plt.xlabel('Stock Price')
#plt.legend()
ax1.legend()
plt.show()
plt.close()
    
print("Mean absolute error of {0}: {1}".format(model_lin,mean_abs_error))

From the above, we can see that the model is trained and evaluated using `LinearRegression`. The predicted and actual adjustable stock prices plots have variations but with a mean absolute error is 0.215 which abit worst than the LSTM model. However, there is still room for improvement and trying out other models to compare. Let't try out another model and see how it performs.

#### Predicting  using Random Forest Regressor

In [ ]:
# Building the model for Adj close prediction

y = np.log(msft_data['Adj Close'].astype(int)) # we want to predict the adjusted close price
X = msft_data.drop('Adj Close', axis=1) # predictive variables (removing Adj close from it)

#split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("x_train", X_train.shape)
print("y_train", y_train.shape)
print("x_test", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
#creating an instance of a Random Forest Regressor 
model_rf = RandomForestRegressor(n_estimators=100, random_state=47)

In [ ]:
#fit the model with the training data
model_rf.fit(X_train,y_train)

In [ ]:
#prediction
predict = model_rf.predict(X_test)
predict #view some predictions

In [ ]:
#calculation of the mean absolute error
mean_abs_error = mean_absolute_error(y_test, predict)
mean_abs_error

In [ ]:
# view predictions and actual values
#print(np.c_[y_test,predict])
display_ = pd.DataFrame({'Actual value': y_test, 'Predicted value':predict})
display_.head(10)
#print(y_test,predict)

From the table, we can see that the `RandomForestRegressor` performed very well and the actual and predicted Adjusted close value are fairly close. Let us now view the plots.

In [ ]:
#plot showing the prediction and actual values

fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=80)
axs[0].set_title('Predicted vs actual values distribution')
ax1 = sns.kdeplot(data=y_test, color="g", label='Actual values',ax=axs[0])
ax2 = sns.kdeplot(data=predict, color="b", label='Predicted values', ax=ax1)
    
sns.regplot(x=y_test, y=predict)
plt.title('Predicted vs actual values distribution')
plt.xlabel('Stock Price')
#plt.legend()
ax1.legend()
plt.show()
plt.close()
    
print("Mean absolute error of {0}: {1}".format(model_rf,mean_abs_error))

From the above, we can see that the model is trained and evaluated using `RandomForestRegressor`. The predicted and actual adjustable stock prices plots look are relatively similar with a mean absolute error is 0.0494 which is good. Let us see how our three models would perfomr with the Google stocks.

### Predicting Adjusted close value of Google stocks

In [ ]:
# get Google data and view the first 5 rows
googl_data = get_data(fin_data, 'GOOGL')  
googl_data.head()

In [ ]:
# plot showing Google historical Adjusted closing prices
plt.figure(figsize=(16,6))
plt.title('Google Adjusted Close Price History')
plt.plot(googl_data['Adj Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Adjusted Close Price USD ($)', fontsize=18)
plt.show()

In [ ]:
# Building the model for Adj close prediction

# create the variables for prediction and split into training and test sets

y = np.log(googl_data['Adj Close'].astype(int)) # we want to predict the adjusted close price
X = googl_data.drop('Adj Close', axis=1) # predictive variables (removing Adj close from it)

#split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("x_train", X_train.shape)
print("y_train", y_train.shape)
print("x_test", X_test.shape)
print("y_test", y_test.shape)

#### Prediction Using LSTM

In [ ]:
#### LSTM Model for Google stocks

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model part 5 (no activation function)
model.fit(np.array(X_train).reshape(-1,5,1), y_train, batch_size =800, epochs=50, verbose=0)

#predictions 
predictions = model.predict(np.array(X_test).reshape(-1,5,1))

#calculation of the mean absolute error
mean_abs_error = mean_absolute_error(y_test, predictions)
mean_abs_error

#### Prediction Using Linear Regression

In [ ]:
#creating an instance of a Linear Regressor 
model_lin2 = LinearRegression()

#fit the model
model_lin2.fit(X_train,y_train)

#prediction
predict = model_lin2.predict(X_test)

#calculation of the mean absolute error
mean_abs_error = mean_absolute_error(y_test, predict)
mean_abs_error

#### Prediction Using Random Forest Regressor

In [ ]:
#creating an instance of a Random Forest Regressor 
model_rf2 = RandomForestRegressor(n_estimators=100, random_state=47)

#fit the model with the training data
model_rf2.fit(X_train,y_train)

#prediction
predict2 = model_rf2.predict(X_test)
predict2 #view some predictions

#calculation of the mean absolute error
mean_abs_error2 = mean_absolute_error(y_test, predict2)
mean_abs_error2

In [ ]:
#plot showing the prediction and actual values

fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=80)
axs[0].set_title('Predicted vs actual values distribution')
ax1 = sns.kdeplot(data=y_test, color="g", label='Actual values',ax=axs[0])
ax2 = sns.kdeplot(data=predict2, color="b", label='Predicted values', ax=ax1)
    
sns.regplot(x=y_test, y=predict2)
plt.title('Predicted vs actual values distribution')
plt.xlabel('Stock Price')
#plt.legend()
ax1.legend()
plt.show()
plt.close()
    
print("Mean absolute error of {0}: {1}".format(model_rf2,mean_abs_error2))

In [ ]:
#plot showing the prediction and actual values

fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=80)
axs[0].set_title('Actual values distribution')
axs[1].set_title('Predicted values distribution')
ax1 = sns.kdeplot(data=y_test, color="g", label='Actual values',ax=axs[0])
ax2 = sns.kdeplot(data=predict2, color="b", label='Predicted values', ax=axs[1])

ax1.legend()
ax2.legend()
plt.show()
plt.close()
    
print("Mean absolute error of {0}: {1}".format(model_rf2,mean_abs_error2))

From the above, we can see that the model is trained and evaluated using `RandomForestRegressor`. The predicted and actual adjustable Google stock prices plots look are very similar with a mean absolute error is 0.000824 which is very good. We can also see that all three models perform better with Google than Microsoft stocks.

## 4. Results

### Evaluation and Validation of models
From my investigation of three different models, I observed that RandomForestRegressor delivered a much lower mean absolute error (0.0497, 0.000824) than the LSTM (0.0942, 0.00646) or LinearRegression (0.215, 0.167) for Microsoft and Google respectively.  I also observed that tunning the parameters for LSTM (e.g the number of epochs and batch_size) resulted in better prediction.

### Interesting facts about the project
When exploring the data, it was interesting to see how the stock prices of different companies changed due to the pandemic and how the technological companies stock prices bounced back more quickly than the other companies considered. It was also interesting to see how Pfizer stocks improved as the vaccine rollout began.

Here are some major highlights from the data exploration section:

  1. 2019: Before the pandemic, most of the companies stocks were doing relatively well with Apple and Microsoft taking the lead and Pfizer trailing behind.
  2. 2020: On the onset of the pandemic around Spring, there was a fall in stock prices for all the companies, but afterwards the technology companies like Amazon, Apple, Microsoft and Google started to grow again. But companies like Pfizer, Ford and S&P 500 did not do very well especially Ford.
  3. 2021: As the vaccine rollout began and the lockdown began to be lifted, there was significant growth in the stock prices of Ford in particular given its very stock prices which was low in 2020 due to the pandemic. Companies like Google and Microsoft,S&P 500 also grew. Overall there was an improvement in the stock prices of all the companies we considered.

### Difficulties encountered
1. Getting the data from yahoo fincance wasn't very obvious. After several research I was able to find an [article](https://towardsdatascience.com/free-stock-data-for-python-using-yahoo-finance-api-9dafd96cad2e) that guided me on how to obtain the data.
2. The [Machine Learning for Trading course](https://www.udacity.com/course/machine-learning-for-trading--ud501) was also really helpful to knowing more about financial data and how to handle it as well as knowing where to begin my data exploration.
3. In the modelling part (particularly the LSTM), one would have to spend some time tuning the parameters and training it  to get the best results.

### Improvements
An improvement to my analysis could be the following:
1. Take some significant time to tune the model parameters as well as include more features that might be relevant for stock price prediction.
2. Try out more models and see if there might be one with a better performance than the RandomForestRegressor. I only tried three models for simplicity and time constraints. 
3. Explore other companies stocks to see how well one can predict their stocks prices with different models.